# Project Day 4 - Data Collection Using Twitter API.


In [2]:
!pip install tweepy

In [3]:
!pip install openpyxl

In [4]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [5]:
consumer_key = "LzsCOl12ZZ79FObx7BV9a1R3a"
consumer_secret = "gODxZ1kHJHDGgYfyP12AUN37UTlDd6yn3OsSV9Tl0iwu0K0cRY"
access_token = "296102538-kYIK8SULCmRK4vgvYD2DhKbwvSXrgTdTkWh0Nvsp"
access_token_secret = "Ib9obwgcE5tZk4S6tY5rAqI5iXOeJc9FGFg28qHoa4YpP"

In [6]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [29]:
query = "MI winning IPL"

In [30]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )


for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x000001D5154911F0>, _json={'created_at': 'Sat Oct 31 10:03:53 +0000 2020', 'id': 1322479398235168768, 'id_str': '1322479398235168768', 'full_text': '#MIvsDC #IPL2020 #DCvsMI\n\nMatch 51, DC vs MI |\n\nOUT! Trent Boult gets Shikhar Dhawan. DC 1/1\n\nFollow live:\n\nhttps://t.co/LjJju2zeZ7', 'truncated': False, 'display_text_range': [0, 131], 'entities': {'hashtags': [{'text': 'MIvsDC', 'indices': [0, 7]}, {'text': 'IPL2020', 'indices': [8, 16]}, {'text': 'DCvsMI', 'indices': [17, 24]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/LjJju2zeZ7', 'expanded_url': 'https://www.timesnownews.com/sports/cricket/article/delhi-capitals-vs-mumbai-indians-ipl-2020-live-score-and-updates-dc-aim-to-return-to-winning-ways/675443', 'display_url': 'timesnownews.com/sports/cricket…', 'indices': [108, 131]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter 

# Giving Attributes to the Tweets

In [9]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [10]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [11]:
stream(data=["MI winning IPL"] , file_name='my_tweets')

In [12]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,Who is winning today #MI or #DC?\nmy predictio...,Manish Bhatt,83220,648,"New Delhi, India",False,1,0,2020-10-31 09:19:27
1,"100% Winning Team, Get Today Match Prediction ...",Veena,66,8,Jaipur,False,0,0,2020-10-31 09:01:38
2,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,The sports 360,540,9,,False,0,2,2020-10-31 08:15:10
3,IPL Breaking for tdy Delh...,I knw Pridiction..,5759,420,India,False,3,0,2020-10-31 08:08:28
4,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,IPL Crickinfo,233,0,,False,0,2,2020-10-31 07:32:45


# Cleaning and Identifying the Sentiments of the Tweets

In [13]:
!pip install textblob

In [14]:
from textblob import TextBlob

In [15]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [16]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [17]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [18]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,Who is winning today #MI or #DC?\nmy predictio...,Manish Bhatt,83220,648,"New Delhi, India",False,1,0,2020-10-31 09:19:27,Who is winning today MI or DC my prediction go...
1,"100% Winning Team, Get Today Match Prediction ...",Veena,66,8,Jaipur,False,0,0,2020-10-31 09:01:38,100 Winning Team Get Today Match Prediction Re...
2,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,The sports 360,540,9,,False,0,2,2020-10-31 08:15:10,RT MIvsDC IPL2020 PollOfTheDay Match 5 1 Who h...
3,IPL Breaking for tdy Delh...,I knw Pridiction..,5759,420,India,False,3,0,2020-10-31 08:08:28,IPL Breaking for tdy Delhi Capitals Winning 1s...
4,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,IPL Crickinfo,233,0,,False,0,2,2020-10-31 07:32:45,RT MIvsDC IPL2020 PollOfTheDay Match 5 1 Who h...


In [19]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,Who is winning today #MI or #DC?\nmy predictio...,Manish Bhatt,83220,648,"New Delhi, India",False,1,0,2020-10-31 09:19:27,Who is winning today MI or DC my prediction go...,Positive
1,"100% Winning Team, Get Today Match Prediction ...",Veena,66,8,Jaipur,False,0,0,2020-10-31 09:01:38,100 Winning Team Get Today Match Prediction Re...,Positive
2,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,The sports 360,540,9,,False,0,2,2020-10-31 08:15:10,RT MIvsDC IPL2020 PollOfTheDay Match 5 1 Who h...,Positive
3,IPL Breaking for tdy Delh...,I knw Pridiction..,5759,420,India,False,3,0,2020-10-31 08:08:28,IPL Breaking for tdy Delhi Capitals Winning 1s...,Positive
4,RT @sportstigerapp: #MIvsDC | #IPL2020 | #Poll...,IPL Crickinfo,233,0,,False,0,2,2020-10-31 07:32:45,RT MIvsDC IPL2020 PollOfTheDay Match 5 1 Who h...,Positive


In [20]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 100% Winning Team, Get Today Match Prediction Report By Us #DuniyaBoliLagayegi #KiaraAdvani #MIvsDC #DCvsMI… https://t.co/LNFs1mASHZ

Clean tweet:
 100 Winning Team Get Today Match Prediction Report By Us DuniyaBoliLagayegi KiaraAdvani MIvsDC DCvsMI

Sentiment of the tweet:
 Positive


In [32]:
n = 45
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Shebas_10dulkar: In 2020 IPL

MI won 5 Consecutive Matches
KXIP Broken that winning Streak

KXIP won 5 Consecutive Matches
RR Broken th…

Clean tweet:
 RT 10dulkar In 2020 IPL MI won 5 Consecutive Matches KXIP Broken that winning Streak KXIP won 5 Consecutive Matches RR Broken th

Sentiment of the tweet:
 Negative


In [22]:
n = 203
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 @unacademy #UnacademyAskTheExperts  This IPL Table has so many possibilities. It is just possible that 5 teams (MI,… https://t.co/gDCjmEYe1w

Clean tweet:
 UnacademyAskTheExperts This IPL Table has so many possibilities It is just possible that 5 teams MI

Sentiment of the tweet:
 Positive


In [23]:
df[df.Sentiment == 'Positive'].shape[0]

442

In [24]:
df[df.Sentiment == 'Neutral'].shape[0]

20

In [25]:
df[df.Sentiment == 'Negative'].shape[0]

70